In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
import pandas as pd
from torch.autograd import Variable

In [4]:
class CustomDatasetFromImages(Dataset):
    def __init__(self, csv_path):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            transform: transform 操作
        """
        # Transforms
        self.to_tensor = transforms.ToTensor()
        # 读取 csv 文件
        self.data_info = pd.read_csv(csv_path, header=None)
        # 文件第一列包含图像文件的名称
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[:, 1],dtype=np.float64)

        maxnum = np.max(self.label_arr)
        minnum = np.min(self.label_arr)
        self.label_arr-=minnum
        self.label_arr/=maxnum-minnum

        # 计算 length
        self.data_len = len(self.data_info.index)
 
    def __getitem__(self, index):
        # 从 pandas df 中得到文件名
        single_image_name = self.image_arr[index]
        # 读取图像文件
        img_as_img = Image.open(single_image_name)
 
        # 检查需不需要额外操作
        
        # 把图像转换成 tensor
        img_as_tensor = self.to_tensor(img_as_img)
 
        # 得到图像的 label
        single_image_label = self.label_arr[index]
 
        return (img_as_tensor, single_image_label)
 
    def __len__(self):
        return self.data_len

In [5]:
if __name__ == "__main__":
    custom_mnist_from_images =  CustomDatasetFromImages('/home/macin/data.csv')
    print(custom_mnist_from_images)
    mn_dataset_loader = torch.utils.data.DataLoader(dataset=custom_mnist_from_images,
                                                    batch_size=10,
                                                    shuffle=False)

In [6]:
for images, labels in mn_dataset_loader:
        print(labels)

tensor([0.8095, 0.8095, 0.8095, 0.8095, 0.8095, 0.8571, 0.9524, 1.0000, 0.9524,
        0.9048], dtype=torch.float64)
tensor([0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571,
        0.8571], dtype=torch.float64)
tensor([0.6667, 0.5714, 0.4762, 0.3333, 0.1905, 0.1071, 0.0833, 0.0833, 0.0714,
        0.0714], dtype=torch.float64)
tensor([0.0714, 0.0595, 0.0595, 0.0595, 0.0595, 0.5238, 0.3333, 0.1667, 0.2143,
        0.1429], dtype=torch.float64)
tensor([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0476,
        0.0476], dtype=torch.float64)
tensor([0.0476, 0.0476, 0.1310, 0.1310, 0.1310, 0.3333, 0.3333, 0.3333, 0.3333,
        0.3333], dtype=torch.float64)
tensor([0.3333, 0.3333, 0.3333, 0.1429, 0.0952, 0.0833, 0.0714, 0.0714, 0.0714,
        0.0714], dtype=torch.float64)
tensor([0.1310, 0.1071, 0.1071, 0.1071, 0.1071, 0.1071, 0.2143, 0.1190, 0.0833,
        0.0595], dtype=torch.float64)
tensor([0.0595, 0.0595, 0.3333, 0.2857, 0.1667, 0.1667, 

In [7]:
class CNNnet(torch.nn.Module):
    def __init__(self):
        super(CNNnet,self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,
                            out_channels=16,
                            kernel_size=3,
                            stride=2,
                            padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU()
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(16,32,3,2,1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU()
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,3,2,1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU()
        )
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv2d(64,64,2,2,0),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU()
        )
        self.mlp1 = torch.nn.Linear(14976,100)
        self.mlp2 = torch.nn.Linear(100,1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.mlp1(x.view(x.size(0),-1))
        x = self.mlp2(x)

        
        return x
model = CNNnet()
print(model)

CNNnet(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (mlp1): Linear(in_features=14976, out_features=100, bias=True)
  (mlp2): Linear(in_features=100, out_features=1, bias=True)
)


In [8]:
loss_func = torch.nn.functional.mse_loss
opt = torch.optim.Adam(model.parameters(),lr=0.001)

In [9]:
loss_count = []
mn_dataset_loader = torch.utils.data.DataLoader(dataset=custom_mnist_from_images,
                                                    batch_size=10,
                                                    shuffle=True)
for epoch in range(10):
    
    i = 0
    for x,y in mn_dataset_loader:
        i = i + 1
        batch_x = x # torch.Size([128, 1, 28, 28])
        batch_y = y # torch.Size([128])
        # 获取最后输出
        out = model(batch_x) # torch.Size([128,10])
        #out = out.view((10,))
        # 获取损失
        batch_y=batch_y.to(torch.float32)
        batch_y=batch_y.view((-1,1))
        loss = loss_func(out,batch_y)
        # 使用优化器优化损失
        opt.zero_grad()  # 清空上一步残余更新参数值
        loss.backward() # 误差反向传播，计算参数更新值
        opt.step() # 将参数更新值施加到net的parmeters上
        if i%10 == 0:
            loss_count.append(loss)
            print('{}:\t'.format(i), loss.item())
            torch.save(model,r'/home/macin/log_CNN')
        if i % 100 == 0:
            for a,b in test_loader:
                test_x = Variable(a)
                test_y = Variable(b)
                out = model(test_x)
                # print('test_out:\t',torch.max(out,1)[1])
                # print('test_y:\t',test_y)
                accuracy = torch.max(out,1)[1].numpy() == test_y.numpy()
                print('accuracy:\t',accuracy.mean())
                break
plt.figure('PyTorch_CNN_Loss')
plt.plot(loss_count,label='Loss')
plt.legend()
plt.show()

/home/macin/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


10:	 42.100364685058594
20:	 22.512548446655273
30:	 0.9314725995063782
40:	 2.199154853820801
10:	 0.26830053329467773
20:	 0.03335753083229065
30:	 0.0937005877494812
40:	 0.06580941379070282
10:	 0.06916395574808121
20:	 0.07257124781608582
30:	 0.04602562636137009
40:	 0.031779684126377106
10:	 0.020391052588820457
20:	 0.013563369400799274
30:	 0.006471426226198673
40:	 0.016502203419804573
10:	 0.02200811170041561
20:	 0.010885282419621944
30:	 0.0011684149503707886
40:	 0.028438320383429527
10:	 0.016581136733293533
20:	 0.010697846300899982
30:	 0.00961343850940466
40:	 0.058981239795684814
10:	 0.01724179834127426
20:	 0.0166944433003664
30:	 0.0131314592435956
40:	 0.009469149634242058
10:	 0.004274834878742695
20:	 0.009566685184836388
30:	 0.058617472648620605
40:	 0.07734418660402298
10:	 0.036832842975854874
20:	 0.022578848525881767
30:	 0.01599990762770176
40:	 0.08224707841873169
10:	 0.025118499994277954
20:	 0.011998948641121387
30:	 0.018629370257258415


KeyboardInterrupt: 